#   IAU Zadanie - III. fáza

### Autori: Peter Brandajsky - 50%, Frederik Duvač - 50%

#### Kedze bolo predosle zadanie chaoticky napisane ohladom toho co mame exportnut, tak sme upravili export kde transformujeme aj testovaci dataset pomocou nasej pipeline. Upraveny kod vyzeral takto:


In [ ]:
'''
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer, PowerTransformer, StandardScaler, MinMaxScaler, FunctionTransformer
import pandas as pd

# Definícia stĺpcov
normal_columns = [
    'c.android.gm', 'c.android.chrome', 'c.dogalize', 'c.katana',
    'c.android.youtube', 'p.android.documentsui', 'p.system',
    'p.android.chrome', 'p.android.externalstorage', 'p.android.gm',
    'p.android.packageinstaller'
]
skewed_column = ['p.olauncher']

X_train = train_data.drop(columns=['mwra'])
y_train = train_data['mwra']
X_test = test_data.drop(columns=['mwra'])
y_test = test_data['mwra']

# Krok 1: Quantile Transformácia pre šikmý stĺpec
quantile_transformer = Pipeline(steps=[
    ('transform', make_column_transformer(
        (QuantileTransformer(output_distribution="normal", random_state=42), skewed_column),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Krok 2: Power Transformácia pre normálne stĺpce
power_transformer = Pipeline(steps=[
    ('transform', make_column_transformer(
        (PowerTransformer(method='yeo-johnson', standardize=True), normal_columns),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Krok 3: Standard Scaling pre normálne stĺpce
standard_scaler = Pipeline(steps=[
    ('transform', make_column_transformer(
        (StandardScaler(), normal_columns),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Krok 4: Min-Max Scaling pre šikmý stĺpec
minmax_scaler = Pipeline(steps=[
    ('transform', make_column_transformer(
        (MinMaxScaler(), skewed_column),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Vytvorenie hlavnej Pipeline s každým krokom zvlášť
pipeline = Pipeline(steps=[
    ('quantile', quantile_transformer),
    ('power', power_transformer),
    ('scaler', standard_scaler),
    ('minmax', minmax_scaler),
], verbose=True)

# Aplikácia pipeline na trénovacie dáta bez 'mwra'
X_train_transformed = pipeline.fit_transform(X_train)
X_test_transformed = pipeline.transform(X_test)

# Spojenie transformovaných dát s cieľovým stĺpcom 'mwra'
train_transformed = pd.concat([y_train.reset_index(drop=True), X_train_transformed], axis=1)
test_transformed = pd.concat([y_test.reset_index(drop=True), pd.DataFrame(X_test_transformed, columns=X_test.columns)], axis=1)

train_transformed
test_transformed

train_transformed.to_csv('dataset/train_transformed.csv', sep='\t', index=False, encoding='utf-8')
test_transformed.to_csv('dataset/test_transformed.csv', sep='\t', index=False, encoding='utf-8')

train_data.to_csv('dataset/train_data.csv', sep='\t', index=False, encoding='utf-8')
test_data.to_csv('dataset/test_data.csv', sep='\t', index=False, encoding='utf-8')
'''

# Načítanie údajov z datasetu


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

train_data = pd.read_csv('dataset/train_transformed.csv', sep='\t')
test_data = pd.read_csv('dataset/test_transformed.csv', sep='\t')

In [ ]:
print("Train Data:")
print(train_data.head())
print("\nTest Data:")
print(test_data.head())

# Kontrola rozmerov
print(f"Train Data Shape: {train_data.shape}")
print(f"Test Data Shape: {test_data.shape}")

In [ ]:
train_data.describe()

In [ ]:
test_data.describe()

In [ ]:
# Rozdelenie na atributy a cielovu premennu
X_train = train_data.drop(columns=['mwra'])
y_train = train_data['mwra']

X_test = test_data.drop(columns=['mwra'])
y_test = test_data['mwra']

# Vyber features s top korelaciou z fazy 2 - moznosti: 'all', 'top_3', 'top_5', 'top_9'
selected_features = 'all'

top_3_features = ['p.android.chrome', 'c.katana', 'p.system']
top_5_features = ['p.android.chrome', 'c.katana', 'p.system', 'c.android.chrome', 'p.android.gm']
top_9_features = ['p.android.chrome', 'c.katana', 'p.system', 'c.android.chrome', 'p.android.gm', 'c.android.youtube',
                  'c.dogalize', 'p.android.externalstorage', 'c.android.gm']

# Prisposobenie X_train a X_test podla vyberu
if selected_features == 'all':
    pass  # Vsetky stlpce v X_train a X_test
elif selected_features == 'top_3':
    X_train = X_train[top_3_features]
    X_test = X_test[top_3_features]
elif selected_features == 'top_5':
    X_train = X_train[top_5_features]
    X_test = X_test[top_5_features]
elif selected_features == 'top_9':
    X_train = X_train[top_9_features]
    X_test = X_test[top_9_features]
else:
    raise ValueError("Neznáma možnosť pre výber features: vyberte 'all', 'top_3' alebo 'top_5'.")

# Vystup informacii o pouzitych vlastnostiach
print("Použité features:", X_train.columns)

# Kontrola rozmerov
print(f"X_train Shape: {X_train.shape}, y_train Shape: {y_train.shape}")
print(f"X_test Shape: {X_test.shape}, y_test Shape: {y_test.shape}")

- Pri možnosti all (12 atribútov), čiže všetkych ktoré sme exportli z fazy 2 sme dostali najlepši výsledok.
- Tieto modely sú schopné zohľadniť viac vzorcov a získať lepšie výsledky, čo znamená, že niektoré z týchto pridaných atribútov poskytujú dôležité informácie, ktoré pomáhajú pri rozlíšení medzi triedami.

In [ ]:
# Kontrola distribucie tried
print("Distribúcia v trénovacom datasete:")
print(y_train.value_counts(normalize=True))

print("\nDistribúcia v testovacom datasete:")
print(y_test.value_counts(normalize=True))

# 3.1 Jednoduchý klasikátor na základe závislosti v dátach

## 3.1.A Implementácia ID3 klasifikátora

In [ ]:
# Rekurzivne rozdelenie dat do podmnozin na zaklade najlepsieho atributu
class DecisionNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value


# Funkcia na vypocet entropie aktualnej datovej mnoziny
def calculate_entropy(y):
    counts = Counter(y)
    total = len(y)
    entropy = -sum((count / total) * np.log2(count / total) for count in counts.values())
    return entropy


# Funkcia na vypocet informacneho zisku pre numericke atributy
def information_gain(X, y, feature):
    parent_entropy = calculate_entropy(y)

    # Pre numericke atributy potrebujeme vypocitat prahove hodnoty
    thresholds = X[feature].sort_values().unique()
    best_gain = 0
    best_threshold = None

    for threshold in thresholds:
        left_y = y[X[feature] <= threshold]
        right_y = y[X[feature] > threshold]

        # Vypocitame vazenu entropiu pre dany prah
        weighted_entropy = (
                (len(left_y) / len(y)) * calculate_entropy(left_y)
                + (len(right_y) / len(y)) * calculate_entropy(right_y)
        )

        gain = parent_entropy - weighted_entropy
        if gain > best_gain:
            best_gain = gain
            best_threshold = threshold

    return best_gain, best_threshold


# Trieda ID3Classifier s podporou numerickych dat
class ID3Classifier:
    def __init__(self, max_depth=2):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y, depth=0):
        if len(set(y)) == 1:
            return DecisionNode(value=y.iloc[0])

        if depth == self.max_depth:
            return DecisionNode(value=y.mode()[0])

        # Najdeme najlepsi atribut a prah
        best_feature = None
        best_threshold = None
        best_gain = -1

        for feature in X.columns:
            gain, threshold = information_gain(X, y, feature)
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_threshold = threshold

        # Ak neexistuje ziadne delenie, vratime list
        if best_feature is None:
            return DecisionNode(value=y.mode()[0])

        node = DecisionNode(feature=best_feature, threshold=best_threshold)
        left_mask = X[best_feature] <= best_threshold
        right_mask = X[best_feature] > best_threshold

        node.left = self.fit(X[left_mask], y[left_mask], depth + 1)
        node.right = self.fit(X[right_mask], y[right_mask], depth + 1)

        return node

    def predict_one(self, x, node):
        if node.value is not None:
            return node.value

        if x[node.feature] <= node.threshold:
            return self.predict_one(x, node.left)
        else:
            return self.predict_one(x, node.right)

    def predict(self, X):
        return X.apply(lambda x: self.predict_one(x, self.tree), axis=1)


# Trenovanie ID3 modelu
id3 = ID3Classifier(max_depth=3)
id3.tree = id3.fit(X_train, y_train)

# Predikcia na trenovacej a testovacej mnozine
y_train_pred_id3 = id3.predict(X_train)
y_test_pred_id3 = id3.predict(X_test)

print("Train predikcie:\n", y_train_pred_id3)
print("Test predikcie:\n", y_test_pred_id3)

## 3.1.B Vyhodnotenie modelu

In [ ]:
print("Výsledky pre trénovaciu množinu:")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred_id3):.4f}")
print(f"Precision: {precision_score(y_train, y_train_pred_id3, average='binary'):.4f}")
print(f"Recall: {recall_score(y_train, y_train_pred_id3, average='binary'):.4f}")
print("\nPodrobný report:")
print(classification_report(y_train, y_train_pred_id3))

print("\nVýsledky pre testovaciu množinu:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_id3):.4f}")
print(f"Precision: {precision_score(y_test, y_test_pred_id3, average='binary'):.4f}")
print(f"Recall: {recall_score(y_test, y_test_pred_id3, average='binary'):.4f}")
print("\nPodrobný report:")
print(classification_report(y_test, y_test_pred_id3))

### Interpretácia výsledkov pre max depth 3 a 12 atributov:
1. Presnosť (Accuracy):
    - Trénovacia presnosť: 86.53 %
    - Testovacia presnosť: 87.07 %
    - Obidve sú veľmi dobré a pomerne stabilné medzi trénovacím a testovacím datasetom, čo naznačuje, že model dobre generalizuje.

2. Presnosť (Precision):
    - Trénovacia presnosť: 87.59 %
    - Testovacia presnosť: 88.26 %
    - Vysoká presnosť na oboch množinách znamená, že model minimalizuje počet falošných pozitív (t.j. nesprávne klasifikované pozitívne prípady).

3. Recall (Revokovanie):
    - Trénovacia revokácia: 91.46 %
    - Testovacia revokácia: 91.53 %
    - Model má veľmi vysokú recall hodnotu, čo znamená, že zachytáva väčšinu skutočných pozitívnych prípadov, čo je dôležité pre odhaľovanie malvéru.

4. F1-Score:
    - F1-Score (pre triedu 1.0): 0.89 (trénovacia) a 0.90 (testovacia)
    - F1-Score je veľmi dobrá metrika pre nevyvážené dáta, ako je náš prípad, kde jedna trieda (1.0) je viac zastúpená. Tento výsledok ukazuje, že model má veľmi dobrý kompromis medzi presnosťou a recallom.

5. Macro a weighted average:
    - Macro avg (priemer medzi triedami):
        - Trénovanie: Precision 0.86, Recall 0.85
        - Testovanie: Precision 0.87, Recall 0.86
    - Weighted avg (vážený priemer podľa počtu príkladov v každej triede):
        - Trénovanie: Precision 0.86, Recall 0.87
        - Testovanie: Precision 0.87, Recall 0.87
    - Tieto hodnoty tiež naznačujú, že model je vyvážený medzi oboma triedami, bez výrazných sklonov.

## 3.1.C Overenie overfittingu

In [ ]:
# Metriky pre trenovaciu a testovaciu mnozinu
train_metrics = [accuracy_score(y_train, y_train_pred_id3), precision_score(y_train, y_train_pred_id3),
                 recall_score(y_train, y_train_pred_id3)]
test_metrics = [accuracy_score(y_test, y_test_pred_id3), precision_score(y_test, y_test_pred_id3),
                recall_score(y_test, y_test_pred_id3)]

metrics = ['Accuracy', 'Precision', 'Recall']

x = np.arange(len(metrics))
width = 0.35

ficomp_g, ax = plt.subplots(figsize=(8, 6))

# Stlpce pre trenovaciu a testovaciu mnozinu
rects1 = ax.bar(x - width / 2, train_metrics, width, label='Train', color='b')
rects2 = ax.bar(x + width / 2, test_metrics, width, label='Test', color='g')

# Pridanie popisov, titulkov a zobrazenie
ax.set_ylabel('Scores')
ax.set_title('Comparison of metrics between Train and Test data')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()

plt.tight_layout()
plt.show()

#### Naše výsledky sú veľmi dobré! Máme vysokú presnosť, recall aj F1-skóre, ktoré naznačujú, že model funguje veľmi dobre na detekciu mwra aktivity. Z interpretácie výsledkov a z grafu vidíme, že metriky sú vyvážené a pre testovaciu množinu sú o niečo lepšie, preto náš model nie je preučený.

# 3.2 Trénovanie a vyhodnotenie klasikátorov strojového učenia

## 3.2.A Trénovanie stromového algoritmu - Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(max_depth=3, random_state=42, criterion='entropy')

# model training
dt_clf.fit(X_train, y_train)

# prediction on test and train set
y_train_pred_dt = dt_clf.predict(X_train)
y_test_pred_dt = dt_clf.predict(X_test)

# performance evaluation
print("Decision Tree Classifier - Trénovacia množina:")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred_dt):.4f}")
print(f"Precision: {precision_score(y_train, y_train_pred_dt, average='binary'):.4f}")
print(f"Recall: {recall_score(y_train, y_train_pred_dt, average='binary'):.4f}")
print("\nPodrobný report:")
print(classification_report(y_train, y_train_pred_dt))

print("Decision Tree Classifier - Testovacia množina:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_dt):.4f}")
print(f"Precision: {precision_score(y_test, y_test_pred_dt, average='binary'):.4f}")
print(f"Recall: {recall_score(y_test, y_test_pred_dt, average='binary'):.4f}")
print("\nPodrobný report:")
print(classification_report(y_test, y_test_pred_dt))

## 3.2.B Trénovanie nestromového algoritmu - Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(random_state=42, max_iter=1000)

# model training
lr_clf.fit(X_train, y_train)

# prediction on test and train set
y_train_pred_lr = lr_clf.predict(X_train)
y_test_pred_lr = lr_clf.predict(X_test)

# performance evaluation
print("Logistic Regression - Trénovacia množina")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred_lr):.4f}")
print(f"Precision: {precision_score(y_train, y_train_pred_lr, average='binary'):.4f}")
print(f"Recall: {recall_score(y_train, y_train_pred_lr, average='binary'):.4f}")
print("\nPodrobný report:")
print(classification_report(y_train, y_train_pred_lr))

print("Logistic Regression - Testovacia množina")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_lr):.4f}")
print(f"Precision: {precision_score(y_test, y_test_pred_lr, average='binary'):.4f}")
print(f"Recall: {recall_score(y_test, y_test_pred_lr, average='binary'):.4f}")
print("\nPodrobný report:")
print(classification_report(y_test, y_test_pred_lr))

## 3.2.C Porovnanie s ID3 klasifikátorom na základe Classification Report pre jednotlive triedy 0 a 1

- Pre triedu 1 (pozitívnu) všetky modely dosahujú veľmi podobné výsledky, Logistic Regression disponuje mierne lepším výkonom.
- Pre triedu 0 (negatívnu) Logistic Regression opäť vykazuje lepšie výsledky, najmä čo sa týka precision.

- Toto porovnanie ukazuje, že Logistic Regression má lepší výkon na oboch triedach v porovnaní s ID3 a Decision Tree. ID3 a Decision Tree majú podobné výsledky, ale Logistic Regression je silnejší vo väčšine metrikách.

#### Podrobnejšie metriky a vizualizácia je v bode 3.2.5

## 3.2.D Vizualizácia pravidiel stromového modelu a Logistic Regression

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
tree.plot_tree(dt_clf, filled=True, feature_names=X_train.columns, class_names=['0', '1'], fontsize=12)
plt.title("Decision Tree Visualization")
plt.show()

#### Niektoré vlastnosti, ako documentsui, externalstorage, alebo gm, majú veľký vplyv na rozhodovanie modelu. Sú na vyšších úrovniach stromu, pretože najlepšie rozlišujú medzi triedami.

In [ ]:
# coefficient extraction a sort by influence
coefficients = pd.DataFrame({'Feature': X_train.columns, 'Coefficient': lr_clf.coef_[0]})
coefficients = coefficients.sort_values(by='Coefficient', ascending=False)

# coefficient visualisation
plt.figure(figsize=(10, 6))
coefficients.plot(kind='bar', x='Feature', y='Coefficient', legend=False, color='skyblue')
plt.title('Logistic Regression Coefficients')
plt.ylabel('Coefficient Value')
plt.tight_layout()
plt.show()

## 3.2.E Vyhodnotenie pomocou Accuracy, Precision, Recall

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Metriky pre každý model
metrics_train = {
    'Model': ['ID3', 'Decision Tree', 'Logistic Regression'],
    'Accuracy': [
        accuracy_score(y_train, y_train_pred_id3),
        accuracy_score(y_train, y_train_pred_dt),
        accuracy_score(y_train, y_train_pred_lr)
    ],
    'Precision': [
        precision_score(y_train, y_train_pred_id3),
        precision_score(y_train, y_train_pred_dt),
        precision_score(y_train, y_train_pred_lr)
    ],
    'Recall': [
        recall_score(y_train, y_train_pred_id3),
        recall_score(y_train, y_train_pred_dt),
        recall_score(y_train, y_train_pred_lr)
    ]
}

metrics_test = {
    'Accuracy': [
        accuracy_score(y_test, y_test_pred_id3),
        accuracy_score(y_test, y_test_pred_dt),
        accuracy_score(y_test, y_test_pred_lr)
    ],
    'Precision': [
        precision_score(y_test, y_test_pred_id3),
        precision_score(y_test, y_test_pred_dt),
        precision_score(y_test, y_test_pred_lr)
    ],
    'Recall': [
        recall_score(y_test, y_test_pred_id3),
        recall_score(y_test, y_test_pred_dt),
        recall_score(y_test, y_test_pred_lr)
    ]
}

# Pretransformovanie do DataFrame
metrics_train_df = pd.DataFrame(metrics_train)
metrics_test_df = pd.DataFrame(metrics_test)

# Vizualizácia všetkých metrík pre trénovacie a testovacie dáta
models = ['ID3', 'Decision Tree', 'Logistic Regression']
metrics = ['Accuracy', 'Precision', 'Recall']

# Nastavenie pre graf
x = np.arange(len(models))
width = 0.15

# Vytvorenie grafu
comp_fig, ax = plt.subplots(figsize=(12, 8))

# Stĺpce
comp_rects1 = ax.bar(x - 2 * width, metrics_train_df['Accuracy'], width, label='Accuracy (Train)', color='green')
comp_rects2 = ax.bar(x - width, metrics_test_df['Accuracy'], width, label='Accuracy (Test)', color='lightgreen')
comp_rects3 = ax.bar(x, metrics_train_df['Precision'], width, label='Precision (Train)', color='skyblue')
comp_rects4 = ax.bar(x + width, metrics_test_df['Precision'], width, label='Precision (Test)', color='lightblue')
comp_rects5 = ax.bar(x + 2 * width, metrics_train_df['Recall'], width, label='Recall (Train)', color='darkred')
comp_rects6 = ax.bar(x + 3 * width, metrics_test_df['Recall'], width, label='Recall (Test)', color='red')

# Pridanie názvov a popisov
ax.set_ylabel('Skóre')
ax.set_title('Porovnanie metrík Accuracy, Precision a Recall medzi modelmi (Trénovacia a Testovacia množina)')
ax.set_xticks(x + width * 0.5)
ax.set_xticklabels(models)
ax.legend(loc='lower right')


# Funkcia pre pridanie hodnôt na stĺpce
def add_labels(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate('%.4f' % height,
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')


# Pridanie hodnôt na stĺpce
add_labels(comp_rects1)
add_labels(comp_rects2)
add_labels(comp_rects3)
add_labels(comp_rects4)
add_labels(comp_rects5)
add_labels(comp_rects6)

# Zobrazenie grafu
plt.tight_layout()
plt.show()

### Výstup:

Z výstupov môžeme vidieť, že Logistic Regression dosiahol najvyššiu presnosť a recall na trénovacej aj testovacej množine. Tento model je najpresnejší a má lepšiu schopnosť zachytiť pozitívne vzory (vysoký recall).

ID3 a Decision Tree majú veľmi podobné výsledky, čo naznačuje, že Decision Tree model (ktorý je optimalizovaný v rámci scikit-learn) poskytuje porovnateľné alebo lepšie výsledky než ručne implementovaný ID3 model. Avšak, medzi ID3 a Decision Tree nie sú výrazné rozdiely, keďže oba modely sú v podstate rozhodovacími stromami.

#### 1. ID3:

- Funguje ako základný model, ktorý má stabilné výsledky, ale mierne zaostáva oproti ostatným modelom.
- Precision, recall a F1-score sú rovnaké pre Decision Tree, čo indikuje podobnú štruktúru.

#### 2. Decision Tree:

- Zlepšenie oproti ID3 v testovacej množine, ale rozdiel je minimálny.
- Lepšia flexibilita algoritmu nevedie k dramatickému zlepšeniu presnosti.

#### 3. Logistic Regression:

- Najvyššie výsledky pre všetky metriky (precision, recall, F1-score) v testovacej množine.
- Model má výhodu v tom, že je vhodný pre tento typ úlohy.

# 3.3 Optimalizácia alias hyperparameter tuning

## 3.3.A

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer, roc_curve

# Parametre pre Random Search s väčším rozsahom
param_dist = {
    'max_depth': [3, 5, 7, 10, 15, 20],              
    'min_samples_split': [2, 5, 10, 15, 20],          
    'min_samples_leaf': [1, 2, 4, 6, 8],            
    'ccp_alpha': [0.0, 0.001, 0.002, 0.003, 0.004]
}

# Vytvorenie scoring dictionary pre všetky metriky
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

# Random Search
print("\nSpúšťam Random Search...")
dt_random = DecisionTreeClassifier(random_state=42)
random_search = RandomizedSearchCV(
    estimator=dt_random,
    param_distributions=param_dist,
    n_iter=100,  # počet náhodných kombinácií na vyskúšanie
    cv=5,
    scoring=scoring,
    refit='accuracy',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

random_search.fit(X_train, y_train)

best_max_depth = random_search.best_params_['max_depth']
best_min_samples_split = random_search.best_params_['min_samples_split']
best_min_samples_leaf = random_search.best_params_['min_samples_leaf']
best_ccp_alpha = random_search.best_params_['ccp_alpha']

print("\nNajlepšie parametre Random Search:")
print(f"max_depth: {best_max_depth}")
print(f"min_samples_split: {best_min_samples_split}")
print(f"min_samples_leaf: {best_min_samples_leaf}")
print(f"ccp_alpha: {best_ccp_alpha}")
print(f"Best score: {random_search.best_score_:.4f}")

# Vyhodnotenie najlepších modelov
best_random_model = random_search.best_estimator_

# Predikcie Random Search
random_train_pred = best_random_model.predict(X_train)
random_test_pred = best_random_model.predict(X_test)
random_train_proba = best_random_model.predict_proba(X_train)[:, 1]
random_test_proba = best_random_model.predict_proba(X_test)[:, 1]

# Graf porovnania metrík
plt.figure(figsize=(12, 6))
metrics = {
    'Accuracy': [
        accuracy_score(y_train, random_train_pred),
        accuracy_score(y_test, random_test_pred)
    ],
    'Precision': [
        precision_score(y_train, random_train_pred),
        precision_score(y_test, random_test_pred)
    ],
    'Recall': [
        recall_score(y_train, random_train_pred),
        recall_score(y_test, random_test_pred)
    ],
    'F1-score': [
        f1_score(y_train, random_train_pred),
        f1_score(y_test, random_test_pred)
    ],
    'ROC AUC': [
        roc_auc_score(y_train, random_train_proba),
        roc_auc_score(y_test, random_test_proba)
    ]
}

x = range(len(metrics))
width = 0.2

plt.bar([i - 3*width/2 for i in x], [m[0] for m in metrics.values()], width, label='Random Train', color='skyblue')
plt.bar([i - width/2 for i in x], [m[1] for m in metrics.values()], width, label='Random Test', color='lightblue')

plt.xlabel('Metriky')
plt.ylabel('Skóre')
plt.title('Porovnanie metrík pre Random Search')
plt.xticks(x, metrics.keys(), rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ROC krivky
plt.figure(figsize=(10, 6))

# Random Search ROC
fpr_random_train, tpr_random_train, _ = roc_curve(y_train, random_train_proba)
fpr_random_test, tpr_random_test, _ = roc_curve(y_test, random_test_proba)
plt.plot(fpr_random_train, tpr_random_train, 'g-', label=f'Random Train (AUC = {roc_auc_score(y_train, random_train_proba):.4f})')
plt.plot(fpr_random_test, tpr_random_test, 'g--', label=f'Random Test (AUC = {roc_auc_score(y_test, random_test_proba):.4f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Krivky pre Random Search')
plt.legend()
plt.grid(True)
plt.show()

print("\nVýsledky Random Search:")
print("\nTrénovacia množina:")
print(f"Accuracy: {accuracy_score(y_train, random_train_pred):.4f}")
print(f"Precision: {precision_score(y_train, random_train_pred):.4f}")
print(f"Recall: {recall_score(y_train, random_train_pred):.4f}")
print(f"F1-score: {f1_score(y_train, random_train_pred):.4f}")
print(f"ROC AUC: {roc_auc_score(y_train, random_train_proba):.4f}")

print("\nTestovacia množina:")
print(f"Accuracy: {accuracy_score(y_test, random_test_pred):.4f}")
print(f"Precision: {precision_score(y_test, random_test_pred):.4f}")
print(f"Recall: {recall_score(y_test, random_test_pred):.4f}")
print(f"F1-score: {f1_score(y_test, random_test_pred):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, random_test_proba):.4f}")

Random Search našiel najlepšie parametre pre Decision Tree Classifier s maximálnou hĺbkou stromu 7, minimálnym počtom vzoriek na rozdelenie 10, minimálnym počtom vzoriek na list 8 a bez orezania (ccp_alpha = 0.0).

Model dosiahol vysokú presnosť (accuracy) na trénovacej množine (0.9049) aj na testovacej množine (0.9029), čo naznačuje, že model nie je pretrénovaný a generalizuje dobre na neznáme dáta.

Ďalšie metriky ako precision, recall, F1-score a ROC AUC sú tiež vysoké, čo naznačuje, že model má dobrý výkon v klasifikácii a je schopný správne identifikovať pozitívne prípady s vysokou presnosťou a citlivosťou.

Celkovo, výsledky naznačujú, že model je dobre nastavený a poskytuje spoľahlivé predikcie na základe zvolených parametrov.

Najlepšie parametre z Random Search budeme ďalej používať v Grid Search nižšie.

In [ ]:
best_params = random_search.best_params_

# Zadefinujeme si parametre pre Grid Search
param_grid_refined = {
    "max_depth": [
        best_params["max_depth"] - 2,
        best_params["max_depth"],
        best_params["max_depth"] + 2,
    ],
    "min_samples_split": [
        best_params["min_samples_split"] - 2,
        best_params["min_samples_split"],
        best_params["min_samples_split"] + 2,
    ],
    "min_samples_leaf": [
        best_params["min_samples_leaf"],
        best_params["min_samples_leaf"] + 1,
    ],
    "ccp_alpha": [
        best_params["ccp_alpha"],
        best_params["ccp_alpha"] + 0.001,
        best_params["ccp_alpha"] + 0.002,
    ],
}

# Grid Search
print("\nSpúšťam Grid Search...")
dt_grid = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(
    estimator=dt_grid,
    param_grid=param_grid_refined,
    cv=5,
    scoring=scoring,
    refit='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

best_max_depth = grid_search.best_params_['max_depth']
best_min_samples_split = grid_search.best_params_['min_samples_split']
best_min_samples_leaf = grid_search.best_params_['min_samples_leaf']
best_ccp_alpha = grid_search.best_params_['ccp_alpha']

print("\nNajlepšie parametre Grid Search:")
print(f"max_depth: {best_max_depth}")
print(f"min_samples_split: {best_min_samples_split}")
print(f"min_samples_leaf: {best_min_samples_leaf}")
print(f"ccp_alpha: {best_ccp_alpha}")
print(f"Best score: {grid_search.best_score_:.4f}")

# Vyhodnotenie najlepších modelov
best_grid_model = grid_search.best_estimator_

# Predikcie Grid Search
grid_train_pred = best_grid_model.predict(X_train)
grid_test_pred = best_grid_model.predict(X_test)
grid_train_proba = best_grid_model.predict_proba(X_train)[:, 1]
grid_test_proba = best_grid_model.predict_proba(X_test)[:, 1]

# Graf porovnania metrík
plt.figure(figsize=(12, 6))
metrics = {
    'Accuracy': [
        accuracy_score(y_train, grid_train_pred),
        accuracy_score(y_test, grid_test_pred)
    ],
    'Precision': [
        precision_score(y_train, grid_train_pred),
        precision_score(y_test, grid_test_pred)
    ],
    'Recall': [
        recall_score(y_train, grid_train_pred),
        recall_score(y_test, grid_test_pred)
    ],
    'F1-score': [
        f1_score(y_train, grid_train_pred),
        f1_score(y_test, grid_test_pred)
    ],
    'ROC AUC': [
        roc_auc_score(y_train, grid_train_proba),
        roc_auc_score(y_test, grid_test_proba)
    ]
}

x = range(len(metrics))
width = 0.2

plt.bar([i - 3*width/2 for i in x], [m[0] for m in metrics.values()], width, label='Grid Train', color='skyblue')
plt.bar([i - width/2 for i in x], [m[1] for m in metrics.values()], width, label='Grid Test', color='lightblue')

plt.xlabel('Metriky')
plt.ylabel('Skóre')
plt.title('Porovnanie metrík pre Grid Search')
plt.xticks(x, metrics.keys(), rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ROC krivky
plt.figure(figsize=(10, 6))

# Grid Search ROC
fpr_grid_train, tpr_grid_train, _ = roc_curve(y_train, grid_train_proba)
fpr_grid_test, tpr_grid_test, _ = roc_curve(y_test, grid_test_proba)

plt.plot(fpr_grid_train, tpr_grid_train, 'r-', label=f'Grid Train (AUC = {roc_auc_score(y_train, grid_train_proba):.4f})')
plt.plot(fpr_grid_test, tpr_grid_test, 'r--', label=f'Grid Test (AUC = {roc_auc_score(y_test, grid_test_proba):.4f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Krivky pre Grid Search')
plt.legend()
plt.grid(True)
plt.show()

print("\nVýsledky Grid Search:")
print("\nTrénovacia množina:")
print(f"Accuracy: {accuracy_score(y_train, grid_train_pred):.4f}")
print(f"Precision: {precision_score(y_train, grid_train_pred):.4f}")
print(f"Recall: {recall_score(y_train, grid_train_pred):.4f}")
print(f"F1-score: {f1_score(y_train, grid_train_pred):.4f}")
print(f"ROC AUC: {roc_auc_score(y_train, grid_train_proba):.4f}")

print("\nTestovacia množina:")
print(f"Accuracy: {accuracy_score(y_test, grid_test_pred):.4f}")
print(f"Precision: {precision_score(y_test, grid_test_pred):.4f}")
print(f"Recall: {recall_score(y_test, grid_test_pred):.4f}")
print(f"F1-score: {f1_score(y_test, grid_test_pred):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, grid_test_proba):.4f}")

Výsledky naznačujú, že model je dobre nastavený a poskytuje spoľahlivé predikcie na základe zvolených parametrov. Porovnaním s výsledkami z Random Search môžeme vidieť, že jemné doladenie parametrov pomocou Grid Search mierne zlepšilo výkon modelu na trénovacej množine, pričom výkon na testovacej množine zostal konzistentný.

## 3.3.B

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Definujeme optimalizované parametre
optimized_params = {
    'max_depth': 7,
    'min_samples_split': 10,
    'min_samples_leaf': 8,
    'ccp_alpha': 0.0
}


# Inicializácia Random Forest s optimalizovanými parametrami
rf = RandomForestClassifier(
    max_depth=optimized_params['max_depth'],
    min_samples_split=optimized_params['min_samples_split'],
    min_samples_leaf=optimized_params['min_samples_leaf'],
    ccp_alpha=optimized_params['ccp_alpha'],
    random_state=42
)

# Trénovanie Random Forest
rf.fit(X_train, y_train)

# Predikcie Random Forest
rf_train_pred = rf.predict(X_train)
rf_test_pred = rf.predict(X_test)
rf_train_proba = rf.predict_proba(X_train)[:, 1]
rf_test_proba = rf.predict_proba(X_test)[:, 1]


# Inicializácia Gradient Boosting s optimalizovanými parametrami
gb = GradientBoostingClassifier(
    max_depth=optimized_params['max_depth'],
    min_samples_split=optimized_params['min_samples_split'],
    min_samples_leaf=optimized_params['min_samples_leaf'],
    ccp_alpha=optimized_params['ccp_alpha'],
    random_state=42,
    n_estimators=100,
    learning_rate=0.025,
    subsample=0.8
)

# Trénovanie Gradient Boosting
gb.fit(X_train, y_train)

# Predikcie Gradient Boosting
gb_train_pred = gb.predict(X_train)
gb_test_pred = gb.predict(X_test)
gb_train_proba = gb.predict_proba(X_train)[:, 1]
gb_test_proba = gb.predict_proba(X_test)[:, 1]

# Graf porovnania metrík
plt.figure(figsize=(12, 6))
metrics = {
    'Accuracy': [
        accuracy_score(y_train, rf_train_pred),
        accuracy_score(y_test, rf_test_pred),
        accuracy_score(y_train, gb_train_pred),
        accuracy_score(y_test, gb_test_pred)
    ],
    'Precision': [
        precision_score(y_train, rf_train_pred),
        precision_score(y_test, rf_test_pred),
        precision_score(y_train, gb_train_pred),
        precision_score(y_test, gb_test_pred)
    ],
    'Recall': [
        recall_score(y_train, rf_train_pred),
        recall_score(y_test, rf_test_pred),
        recall_score(y_train, gb_train_pred),
        recall_score(y_test, gb_test_pred)
    ],
    'F1-score': [
        f1_score(y_train, rf_train_pred),
        f1_score(y_test, rf_test_pred),
        f1_score(y_train, gb_train_pred),
        f1_score(y_test, gb_test_pred)
    ],
    'ROC AUC': [
        roc_auc_score(y_train, rf_train_proba),
        roc_auc_score(y_test, rf_test_proba),
        roc_auc_score(y_train, gb_train_proba),
        roc_auc_score(y_test, gb_test_proba)
    ]
}

x = range(len(metrics))
width = 0.2

plt.bar([i - 3*width/2 for i in x], [m[0] for m in metrics.values()], width, label='RF Train', color='skyblue')
plt.bar([i - width/2 for i in x], [m[1] for m in metrics.values()], width, label='RF Test', color='lightblue')
plt.bar([i + width/2 for i in x], [m[2] for m in metrics.values()], width, label='GB Train', color='lightgreen')
plt.bar([i + 3*width/2 for i in x], [m[3] for m in metrics.values()], width, label='GB Test', color='palegreen')

plt.xlabel('Metriky')
plt.ylabel('Skóre')
plt.title('Porovnanie metrík pre Random Forest a Gradient Boosting')
plt.xticks(x, metrics.keys(), rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ROC krivky
plt.figure(figsize=(10, 6))

# Random Forest ROC
fpr_rf_train, tpr_rf_train, _ = roc_curve(y_train, rf_train_proba)
fpr_rf_test, tpr_rf_test, _ = roc_curve(y_test, rf_test_proba)
plt.plot(fpr_rf_train, tpr_rf_train, 'b-', label=f'RF Train (AUC = {roc_auc_score(y_train, rf_train_proba):.4f})')
plt.plot(fpr_rf_test, tpr_rf_test, 'b--', label=f'RF Test (AUC = {roc_auc_score(y_test, rf_test_proba):.4f})')

# Gradient Boosting ROC
fpr_gb_train, tpr_gb_train, _ = roc_curve(y_train, gb_train_proba)
fpr_gb_test, tpr_gb_test, _ = roc_curve(y_test, gb_test_proba)
plt.plot(fpr_gb_train, tpr_gb_train, 'g-', label=f'GB Train (AUC = {roc_auc_score(y_train, gb_train_proba):.4f})')
plt.plot(fpr_gb_test, tpr_gb_test, 'g--', label=f'GB Test (AUC = {roc_auc_score(y_test, gb_test_proba):.4f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Krivky pre Random Forest a Gradient Boosting')
plt.legend()
plt.grid(True)
plt.show()

# Výpis výsledkov
print("\nVýsledky Random Forest:")
print("\nTrénovacia množina:")
print(f"Accuracy: {accuracy_score(y_train, rf_train_pred):.4f}")
print(f"Precision: {precision_score(y_train, rf_train_pred):.4f}")
print(f"Recall: {recall_score(y_train, rf_train_pred):.4f}")
print(f"F1-score: {f1_score(y_train, rf_train_pred):.4f}")
print(f"ROC AUC: {roc_auc_score(y_train, rf_train_proba):.4f}")

print("\nTestovacia množina:")
print(f"Accuracy: {accuracy_score(y_test, rf_test_pred):.4f}")
print(f"Precision: {precision_score(y_test, rf_test_pred):.4f}")
print(f"Recall: {recall_score(y_test, rf_test_pred):.4f}")
print(f"F1-score: {f1_score(y_test, rf_test_pred):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, rf_test_proba):.4f}")

print("\nVýsledky Gradient Boosting:")
print("\nTrénovacia množina:")
print(f"Accuracy: {accuracy_score(y_train, gb_train_pred):.4f}")
print(f"Precision: {precision_score(y_train, gb_train_pred):.4f}")
print(f"Recall: {recall_score(y_train, gb_train_pred):.4f}")
print(f"F1-score: {f1_score(y_train, gb_train_pred):.4f}")
print(f"ROC AUC: {roc_auc_score(y_train, gb_train_proba):.4f}")

print("\nTestovacia množina:")
print(f"Accuracy: {accuracy_score(y_test, gb_test_pred):.4f}")
print(f"Precision: {precision_score(y_test, gb_test_pred):.4f}")
print(f"Recall: {recall_score(y_test, gb_test_pred):.4f}")
print(f"F1-score: {f1_score(y_test, gb_test_pred):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, gb_test_proba):.4f}")

### 1. Random Forest:


#### Trénovacia množina:

- Accuracy: 0.9151 (91.51%) - veľmi dobrá presnosť
- Vyvážené metriky (Precision: 0.9153, Recall: 0.9527)
- ROC AUC: 0.9364 - výborná klasifikačná schopnosť

#### Testovacia množina:

- Accuracy: 0.9274 (92.74%) - dokonca lepšia ako na trénovacej množine
- Podobné metriky ako na trénovacej množine
- Žiadny overfitting, dokonca mierne lepšie výsledky na teste

### 2. Gradient Boosting:


#### Trénovacia množina:

- Accuracy: 0.9282 (92.82%) - veľmi vysoká presnosť
- Vysoké hodnoty všetkých metrík
- ROC AUC: 0.9699 (Musel som zmenšiť learning rate z 0.1 na 0.025, lebo ROC AUC vychádzali obrovské hodnoty, čo boli znaky overfittingu)


#### Testovacia množina:

- Accuracy: 0.9160 (91.60%), rozdiel 0.122

#### Záver
- Rozdiel medzi train a test pri learning rate 0.1 bol ~4.2% - čo indikovalo overfitting
- Výsledky sú konzistentné medzi trénovacou a testovacou množinou
- Teraz je rozdiel medzi modelmi minimálny, čo naznačuje, že parametre sú dobre optimalizované


## 3.3.C

In [ ]:
from sklearn.model_selection import KFold, cross_validate


def get_scores_cv(model, model_name, X, y, cv):
    metrics = ["accuracy", "precision", "recall", "f1", "roc_auc"]
    # Create MultiIndex DataFrame
    df = pd.DataFrame(
        index=pd.MultiIndex.from_product([["Train", "Test", "Difference"], metrics]),
        columns=pd.MultiIndex.from_product([[model_name], ["Mean", "Std"]]),
    )
    # Calculate scores for each model
    scores = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring={
            "accuracy": "accuracy",
            "precision": "precision",
            "recall": "recall",
            "f1": "f1",
            "roc_auc": "roc_auc",
        },
        return_train_score=True,
    )
    # Fill DataFrame with training and test scores
    for metric in metrics:
        # Training scores
        train_key = f"train_{metric}"
        df.loc[("Train", metric), (model_name, "Mean")] = scores[train_key].mean()
        df.loc[("Train", metric), (model_name, "Std")] = scores[train_key].std()
        # Test scores
        test_key = f"test_{metric}"
        df.loc[("Test", metric), (model_name, "Mean")] = scores[test_key].mean()
        df.loc[("Test", metric), (model_name, "Std")] = scores[test_key].std()
        # Difference between training and test scores
        diff = scores[train_key] - scores[test_key]
        df.loc[("Difference", metric), (model_name, "Mean")] = diff.mean()
        df.loc[("Difference", metric), (model_name, "Std")] = diff.std()
    return df

# Nastavenie cross-validácie
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Definovanie modelov s optimalizovanými parametrami
rf = RandomForestClassifier(
    max_depth=7,
    min_samples_split=10,
    min_samples_leaf=8,
    ccp_alpha=0.0,
    random_state=42
)

gb = GradientBoostingClassifier(
    max_depth=7,
    min_samples_split=10,
    min_samples_leaf=8,
    ccp_alpha=0.0,
    learning_rate=0.025,
    n_estimators=100,
    subsample=0.8,
    random_state=42
)

# Získanie výsledkov cross-validácie
rf_scores = get_scores_cv(rf, "Random Forest", X_train, y_train, cv)
gb_scores = get_scores_cv(gb, "Gradient Boosting", X_train, y_train, cv)

# Spojenie výsledkov
all_scores = pd.concat([rf_scores, gb_scores], axis=1)

# Výpis výsledkov
print("Cross-Validation Results:")
print("-" * 80)
print(all_scores)

# Vizualizácia výsledkov
metrics = ["accuracy", "precision", "recall", "f1", "roc_auc"]
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plot pre Random Forest
rf_train = [rf_scores.loc[('Train', metric), ('Random Forest', 'Mean')] for metric in metrics]
rf_test = [rf_scores.loc[('Test', metric), ('Random Forest', 'Mean')] for metric in metrics]
rf_train_std = [rf_scores.loc[('Train', metric), ('Random Forest', 'Std')] for metric in metrics]
rf_test_std = [rf_scores.loc[('Test', metric), ('Random Forest', 'Std')] for metric in metrics]

x = np.arange(len(metrics))
width = 0.35

axes[0].bar(x - width/2, rf_train, width, yerr=rf_train_std, label='Train', color='skyblue', capsize=5)
axes[0].bar(x + width/2, rf_test, width, yerr=rf_test_std, label='Test', color='lightblue', capsize=5)
axes[0].set_title('Random Forest')
axes[0].set_xticks(x)
axes[0].set_xticklabels(metrics, rotation=45)
axes[0].legend()
axes[0].grid(True)

# Plot pre Gradient Boosting
gb_train = [gb_scores.loc[('Train', metric), ('Gradient Boosting', 'Mean')] for metric in metrics]
gb_test = [gb_scores.loc[('Test', metric), ('Gradient Boosting', 'Mean')] for metric in metrics]
gb_train_std = [gb_scores.loc[('Train', metric), ('Gradient Boosting', 'Std')] for metric in metrics]
gb_test_std = [gb_scores.loc[('Test', metric), ('Gradient Boosting', 'Std')] for metric in metrics]

axes[1].bar(x - width/2, gb_train, width, yerr=gb_train_std, label='Train', color='lightgreen', capsize=5)
axes[1].bar(x + width/2, gb_test, width, yerr=gb_test_std, label='Test', color='palegreen', capsize=5)
axes[1].set_title('Gradient Boosting')
axes[1].set_xticks(x)
axes[1].set_xticklabels(metrics, rotation=45)
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

### 1. Trénovacie výsledky:

Gradient Boosting dosahuje lepšie výsledky na trénovacích dátach vo všetkých metrikách

- Accuracy: 92.94% vs 91.92%
- Precision: 92.70% vs 91.94%
- Recall: 96.34% vs 95.49%
- F1-skóre: 94.48% vs 93.68%
- ROC-AUC: 97.41% vs 94.40%


GBM má tiež mierne nižšiu štandardnú odchýlku, čo indikuje stabilnejšie výsledky

### 2. Testovacie výsledky:

Rozdiel medzi klasifikátormi je menší na testovacích dátach
GBM si udržiava miernu prevahu:

- Accuracy: 91.43% vs 91.25%
- Precision: 91.46% vs 91.45% (takmer identické)
- Recall: 95.23% vs 94.91%
- F1-skóre: 93.31% vs 93.15%
- ROC-AUC: 91.91% vs 91.66%



### 3. Rozdiel medzi trénovacími a testovacími výsledkami (overfitting):

GBM vykazuje väčší rozdiel medzi trénovacími a testovacími výsledkami:

- Accuracy rozdiel: 1.51% vs 0.68%
- Precision rozdiel: 1.24% vs 0.48%
- ROC-AUC rozdiel: 5.51% vs 2.73%


Toto naznačuje, že GBM má väčšiu tendenciu k overfittingu

## 3.3.D

In [ ]:
from sklearn.model_selection import learning_curve

# Inicializácia najlepšieho modelu (Gradient Boosting)
best_model = GradientBoostingClassifier(
    max_depth=7,
    min_samples_split=10,
    min_samples_leaf=8,
    ccp_alpha=0.0,
    learning_rate=0.025,
    n_estimators=100,
    subsample=0.8,
    random_state=42
)
# Výpočet learning curves
train_sizes = np.linspace(0.1, 1.0, 10)
train_sizes, train_scores, val_scores = learning_curve(
    best_model, X_train, y_train,
    train_sizes=train_sizes,
    cv=5,
    n_jobs=-1,
    scoring='accuracy'
)

# Výpočet priemerov a štandardných odchýlok
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

# Vizualizácia learning curves
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label='Training score', color='blue', marker='o')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
plt.plot(train_sizes, val_mean, label='Cross-validation score', color='green', marker='o')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1, color='green')

plt.xlabel('Trénovacia množina veľkosť')
plt.ylabel('Accuracy')
plt.title('Learning Curves - Random Forest')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

# Výpis rozdielov medzi trénovacou a validačnou presnosťou
print("Analýza learning curves:")
print("-" * 50)
for size, train_score, val_score in zip(train_sizes, train_mean, val_mean):
    n_samples = int(size * len(X_train))
    diff = train_score - val_score
    print(f"Počet vzoriek: {n_samples}")
    print(f"Trénovacia presnosť: {train_score:.4f}")
    print(f"Validačná presnosť: {val_score:.4f}")
    print(f"Rozdiel: {diff:.4f}")
    print()

# Kontrola overfittingu
final_train_val_diff = train_mean[-1] - val_mean[-1]
print("Finálna analýza overfittingu:")
print("-" * 50)
print(f"Konečná trénovacia presnosť: {train_mean[-1]:.4f}")
print(f"Konečná validačná presnosť: {val_mean[-1]:.4f}")
print(f"Konečný rozdiel: {final_train_val_diff:.4f}")

# Kritériá pre overenie overfittingu
print("\nKritériá pre overenie overfittingu:")
print("-" * 50)

# 1. Rozdiel medzi trénovacou a validačnou presnosťou
print("1. Rozdiel medzi trénovacou a validačnou presnosťou:")
if final_train_val_diff < 0.05:
    print("✓ Malý rozdiel (< 0.05) medzi trénovacou a validačnou presnosťou")
else:
    print("✗ Veľký rozdiel medzi trénovacou a validačnou presnosťou")

# 2. Stabilita validačnej presnosti
val_stability = np.std(val_mean[-3:])
print("\n2. Stabilita validačnej presnosti:")
if val_stability < 0.01:
    print("✓ Validačná presnosť je stabilná (štandardná odchýlka < 0.01)")
else:
    print("✗ Validačná presnosť nie je stabilná")

# 3. Trend validačnej presnosti
val_trend = val_mean[-1] - val_mean[-3]
print("\n3. Trend validačnej presnosti:")
if abs(val_trend) < 0.01:
    print("✓ Validačná presnosť sa ustálila (zmena < 0.01)")
else:
    print("✗ Validačná presnosť stále nie je ustálená")

print("\nZáverečné zhodnotenie:")
print("-" * 50)
if final_train_val_diff < 0.05 and val_stability < 0.01 and abs(val_trend) < 0.01:
    print("Model NEVYKAZUJE známky overfittingu:")
    print("- Malý rozdiel medzi trénovacou a validačnou presnosťou")
    print("- Stabilná validačná presnosť")
    print("- Ustálený výkon na validačnej množine")
else:
    print("Model môže vykazovať známky overfittingu")

# 3.4 Vyhodnotenie vplyvu zvolenej stratégie riešenia na klasikáciu

## 3.4.A

#### Keďže chýbajúce hodnoty, ktoré sme našli boli z takých datasetov, ktoré ani nepoužívame na trénovanie modelu a nemajú žiadnu koreláciu medzi mrwa, tak nebudeme vyhodnocovať stratégiu nahrádzania týchto hodnôt

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer, PowerTransformer, StandardScaler, MinMaxScaler, FunctionTransformer
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


def handle_outliers_iqr(df, columns):
    df_clean = df.copy()
    outliers_count = {}
    
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Identifikácia outlierov
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column]
        outliers_count[column] = len(outliers)
        
        # Nahradenie outlierov mediánom hodnôt, ktoré nie sú outliers
        non_outlier_median = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)][column].median()
        
        # Nahradenie outlierov mediánom namiesto hraničných hodnôt
        df_clean.loc[df_clean[column] < lower_bound, column] = non_outlier_median
        df_clean.loc[df_clean[column] > upper_bound, column] = non_outlier_median
        
    
    return df_clean, outliers_count

def handle_outliers_zscore(df, columns, threshold=3):
    df_clean = df.copy()
    outliers_count = {}
    
    for column in columns:
        mean = df[column].mean()
        std = df[column].std()
        z_scores = abs((df[column] - mean) / std)
        
        # Identifikácia outlierov
        outliers = df[z_scores > threshold][column]
        outliers_count[column] = len(outliers)
        
        # Nahradenie outlierov mediánom hodnôt, ktoré nie sú outliers
        non_outlier_median = df[z_scores <= threshold][column].median()
        
        # Nahradenie outlierov mediánom namiesto priemeru
        df_clean.loc[z_scores > threshold, column] = non_outlier_median
    
    return df_clean, outliers_count

# 1. Načítanie pôvodných dát
print("Načítavanie dát...")
connections = pd.read_csv('dataset/connections.csv', sep='\t')
processes = pd.read_csv('dataset/processes.csv', sep='\t')

# 2. Príprava troch datasetov (original, IQR, Z-score)
print("\nPríprava datasetov...")

# Spojenie datasetov
data_orig = pd.merge(connections, processes, on=['imei', 'ts'])
data_orig = data_orig.drop(columns='mwra_y')
data_orig = data_orig.rename(columns={'mwra_x': 'mwra'})

# Aplikácia metód na odstránenie outlierov
data_iqr, iqr_outliers = handle_outliers_iqr(data_orig, data_orig.columns[1:])
data_zscore, zscore_outliers = handle_outliers_zscore(data_orig, data_orig.columns[1:])

# odstránenie nevyužiteľných stĺpcov
data_iqr = data_iqr[
    ['mwra', 'c.android.gm', 'c.android.chrome', 'c.dogalize', 'c.katana', 'c.android.youtube', 'p.android.documentsui',
     'p.system', 'p.android.chrome', 'p.android.externalstorage',
     'p.android.gm', 'p.android.packageinstaller', 'p.olauncher']]
data_zscore = data_zscore[
    ['mwra', 'c.android.gm', 'c.android.chrome', 'c.dogalize', 'c.katana', 'c.android.youtube', 'p.android.documentsui',
        'p.system', 'p.android.chrome', 'p.android.externalstorage', 'p.android.gm', 'p.android.packageinstaller',
        'p.olauncher']]
data_orig = data_orig[['mwra', 'c.android.gm', 'c.android.chrome', 'c.dogalize', 'c.katana', 'c.android.youtube',
                        'p.android.documentsui', 'p.system', 'p.android.chrome', 'p.android.externalstorage',
                        'p.android.gm', 'p.android.packageinstaller', 'p.olauncher']]

# 3. Definícia stĺpcov pre transformácie
normal_columns = [
    'c.android.gm', 'c.android.chrome', 'c.dogalize', 'c.katana',
    'c.android.youtube', 'p.android.documentsui', 'p.system',
    'p.android.chrome', 'p.android.externalstorage', 'p.android.gm',
    'p.android.packageinstaller'
]
skewed_column = ['p.olauncher']

# 4. Vytvorenie pipeline pre Gradient Boosting

# rozdelenie IRQ a Z-score dát na trénovacie a testovacie
X_train_iqr, X_test_iqr, y_train_iqr, y_test_iqr = train_test_split(data_iqr.drop(columns='mwra'), data_iqr['mwra'], test_size=0.2, random_state=42)
X_train_zscore, X_test_zscore, y_train_zscore, y_test_zscore = train_test_split(data_zscore.drop(columns='mwra'), data_zscore['mwra'], test_size=0.2, random_state=42)
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(data_orig.drop(columns='mwra'), data_orig['mwra'], test_size=0.2, random_state=42)

# Quantile Transformácia pre šikmý stĺpec
quantile_transformer = Pipeline(steps=[
    ('transform', make_column_transformer(
        (QuantileTransformer(output_distribution="normal", random_state=42), skewed_column),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Power Transformácia pre normálne stĺpce
power_transformer = Pipeline(steps=[
    ('transform', make_column_transformer(
        (PowerTransformer(method='yeo-johnson', standardize=True), normal_columns),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Standard Scaling pre normálne stĺpce
standard_scaler = Pipeline(steps=[
    ('transform', make_column_transformer(
        (StandardScaler(), normal_columns),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Min-Max Scaling pre šikmý stĺpec
minmax_scaler = Pipeline(steps=[
    ('transform', make_column_transformer(
        (MinMaxScaler(), skewed_column),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Vytvorenie hlavnej Pipeline s každým krokom zvlášť
pipeline = Pipeline(steps=[
    ('quantile', quantile_transformer),
    ('power', power_transformer),
    ('scaler', standard_scaler),
    ('minmax', minmax_scaler),
], verbose=True)

# Aplikácia pipeline na z-score a iqr dáta
X_train_iqr_transformed = pipeline.fit_transform(X_train_iqr)
X_test_iqr_transformed = pipeline.transform(X_test_iqr)
X_train_zscore_transformed = pipeline.fit_transform(X_train_zscore)
X_test_zscore_transformed = pipeline.transform(X_test_zscore)
X_train_orig_transformed = pipeline.fit_transform(X_train_orig)
X_test_orig_transformed = pipeline.transform(X_test_orig)


# definovanie metrik pre cross validáciu
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

# Inicializácia Gradient Boosting
gb = GradientBoostingClassifier(
    max_depth=7,
    min_samples_split=10,
    min_samples_leaf=8,
    ccp_alpha=0.0,
    learning_rate=0.025,
    n_estimators=100,
    subsample=0.8,
    random_state=42
)

def print_metrics(y_true, y_pred, name):
    print(f"\n{name}:")
    print("-" * 50)
    print(f"accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"precision: {precision_score(y_true, y_pred):.4f}")
    print(f"recall: {recall_score(y_true, y_pred):.4f}")
    print(f"f1: {f1_score(y_true, y_pred):.4f}")
    print(f"roc_auc: {roc_auc_score(y_true, y_pred):.4f}")
    return accuracy_score(y_true, y_pred)

# Trénovanie a predikcia pre pôvodné dáta
gb.fit(X_train_orig_transformed, y_train_orig)
y_pred_orig = gb.predict(X_test_orig_transformed)
orig_acc = print_metrics(y_test_orig, y_pred_orig, "Bez odstránenia outlierov")

# Trénovanie a predikcia pre IQR dáta
gb.fit(X_train_iqr_transformed, y_train_iqr)
y_pred_iqr = gb.predict(X_test_iqr_transformed)
iqr_acc = print_metrics(y_test_iqr, y_pred_iqr, "IQR")

# Trénovanie a predikcia pre Z-score dáta
gb.fit(X_train_zscore_transformed, y_train_zscore)
y_pred_zscore = gb.predict(X_test_zscore_transformed)
zscore_acc = print_metrics(y_test_zscore, y_pred_zscore, "Z-score")

# Porovnanie zlepsenia
print("\nPorovnanie zlepseni:")
print("-" * 50)
iqr_improvement = ((iqr_acc - orig_acc) / orig_acc) * 100
zscore_improvement = ((zscore_acc - orig_acc) / orig_acc) * 100

print(f"IQR zlepsenie: {iqr_improvement:.2f}%")
print(f"Z-score zlepsenie: {zscore_improvement:.2f}%")

Podľa výsledkov je zrejmé, že pre náš najlepší model, odstraňovanie outlierov nevylepšilo accuracy, ale naopak. Môže to byť tým, že outlier hodnoty možno obsahuju dôležité infromácie pre klasifikáciu alebo to bude tým, že Gradient Boosting je prirodzene robustný voči outlierom.

## 3.4.B

#### 1. Účinnosť transformácií:

Baseline (bez transformácií): ~91.38% accuracy
S transformáciami: ~91.43% accuracy
Zlepšenie: +0.05%

#### 2. Vyhodnotenie efektivity:

- Stabilnejšie výsledky (menšia štandardná odchýlka)
- Lepšia konvergencia modelu
- Robustnosť voči outlierom
- Zlepšená interpretovateľnosť modelu

#### 3. Záver:

- Aj keď zlepšenie accuracy nie je dramatické, transformácie prinášajú iné benefity (stabilita, interpretovateľnosť)
- Pre tento dataset je kombinácia rôznych transformácií vhodná
- Transformácie pomáhajú najmä pri spracovaní šikmých dát a outlierov

Tento prístup k dátovým transformáciám sa ukázal ako primerane účinný pre náš dataset, aj keď priestor na vylepšenie stále existuje.

## 3.4.C

#### 1. Výsledky Random Search:

Najlepšie parametre:
- max_depth=7,
- min_samples_split=10,
- min_samples_leaf=8,
- ccp_alpha=0.0,

Výsledky na testovacích dátach:

- Accuracy: 0.9029
- Precision: 0.9015
- Recall: 0.9512
- F1-score: 0.9257
- ROC AUC: 0.8901



#### 2. Grid Search (jemné doladenie):

- Použili sme zjemnené parametre okolo najlepších hodnôt z Random Search
- Parametre zostali rovnaké ako pri Random Search, čo potvrdzuje ich optimalitu
- Learning rate bol znížený na 0.025 pre lepšiu generalizáciu
- Subsample 0.8 pre redukciu overfittingu

#### 3. Finálny optimalizovaný model (Gradient Boosting):

- max_depth=7,
- min_samples_split=10,
- min_samples_leaf=8,
- ccp_alpha=0.0,
- learning_rate=0.025,
- n_estimators=100,
- subsample=0.8,
- random_state=42

#### 4. Vplyv na vykon

- Pôvodný model bez optimalizácie: ~91.38% accuracy
- Optimalizovaný model: ~91.60% accuracy
- Zlepšenie: +0.22%
- Zlepšenie stability (nižšia variancia medzi trénovacou a testovacou množinou)

#### 5. Learning curves

- Malý rozdiel medzi trénovacou a validačnou presnosťou (<0.05)
- Stabilná validačná presnosť (std < 0.01)
- Ustálený výkon na validačnej množine


## 3.4.D

#### Na základe našich experimentov a analýz môžem identifikovať, že najlepší model pre nasadenie je Gradient Boosting Classifier s configuraciou:

- max_depth=7
- min_samples_split=10
- min_samples_leaf=8
- ccp_alpha=0.0
- learning_rate=0.025
- n_estimators=100
- subsample=0.8
- random_state=42

#### Dovody vyberu:

- Najvyššia presnosť: V porovnaní s ostatnými modelmi (Random Forest, Decision Tree, ID3) dosiahol najlepšie výsledky
- Stabilita: Vykazuje konzistentné výsledky medzi trénovacou a testovacou množinou
- Vyváženosť metrík: Dobré vyváženie medzi precision a recall
- Robustnosť: Prirodzene odolný voči outlierom
- Nízky overfitting: Rozdiel medzi trénovacou a testovacou presnosťou je minimálny


## 3.4.E

#### Data Pipeline v produkcii:

##### 1. Načítanie dát:

- Spájanie connections a processes tabuliek podľa 'imei' a 'ts'
- Odstránenie duplicitného stĺpca 'mwra_y' a premenovanie 'mwra_x' na 'mwra'


##### 2. Feature Selection:

 Ponechanie len relevantných stĺpcov s najvyššou koreláciou:
```
selected_columns = [
    'mwra', 'c.android.gm', 'c.android.chrome', 'c.dogalize', 
    'c.katana', 'c.android.youtube', 'p.android.documentsui',
    'p.system', 'p.android.chrome', 'p.android.externalstorage',
    'p.android.gm', 'p.android.packageinstaller', 'p.olauncher'
]
```
##### 3. Transformácie dát:

##### Pre šikmé dáta (p.olauncher):

- Najprv Quantile transformácia na normálnu distribúciu
- Potom Min-Max scaling


Pre normálne rozdelené stĺpce:

- Power transformácia (Yeo-Johnson)
- Standard scaling

##### 4. Rozdelenie dát:

- Train-test split (80:20)
- Stratifikácia podľa cieľovej premennej 'mwra'

##### 5. Export:

- Uloženie transformovaných trénovacích dát
- Uloženie transformovaných testovacích dát
- Uloženie pôvodných trénovacích a testovacích dát pre referenciu
